In [1]:
import yfinance as yf
import pandas as pd

# Ticker für Microsoft
ticker = "MSFT"
msft = yf.Ticker(ticker)

# Verfügbare Verfallstage
expirations = msft.options

# Leere Listen für alle Daten
all_calls = []
all_puts = []

# Durchlaufe alle Verfallstage
for expiry in expirations:
    try:
        chain = msft.option_chain(expiry)
        calls = chain.calls.copy()
        puts = chain.puts.copy()

        calls["expirationDate"] = expiry
        puts["expirationDate"] = expiry

        all_calls.append(calls)
        all_puts.append(puts)
    except Exception as e:
        print(f"Fehler bei {expiry}: {e}")

# Kombiniere alle Daten
df_calls = pd.concat(all_calls, ignore_index=True)
df_puts = pd.concat(all_puts, ignore_index=True)

# Optional: CSV exportieren
df_calls.to_csv("MSFT_calls.csv", index=False)
df_puts.to_csv("MSFT_puts.csv", index=False)

print("Option Chains wurden gespeichert.")

Verfügbare Verfallstage: ('2025-04-04', '2025-04-11', '2025-04-17', '2025-04-25', '2025-05-02', '2025-05-09', '2025-05-16', '2025-06-20', '2025-07-18', '2025-08-15', '2025-09-19', '2025-10-17', '2025-12-19', '2026-01-16', '2026-03-20', '2026-06-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17')

Calls:
        contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0  MSFT250404C00230000 2025-03-19 19:10:01+00:00   230.0     160.07  141.90   
1  MSFT250404C00250000 2025-03-28 15:30:30+00:00   250.0     133.64  121.00   
2  MSFT250404C00290000 2025-03-28 14:37:11+00:00   290.0      94.15   81.45   
3  MSFT250404C00300000 2025-04-03 18:28:43+00:00   300.0      75.21   71.50   
4  MSFT250404C00310000 2025-04-01 16:29:15+00:00   310.0      71.10   61.50   

      ask    change  percentChange  volume  openInterest  impliedVolatility  \
0  145.25  0.000000        0.00000     1.0             4           3.144533   
1  125.25  0.000000        0.00000     3.0           

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Hole Optionen für MSFT
ticker = "MSFT"
msft = yf.Ticker(ticker)
expirations = msft.options

# Wähle nur Calls, nur die ersten 10 Verfallstage (Performance)
calls_data = []

for expiry in expirations[:10]:
    try:
        chain = msft.option_chain(expiry)
        calls = chain.calls[['strike', 'impliedVolatility']].copy()
        calls['expiration'] = expiry
        calls_data.append(calls)
    except:
        continue

df = pd.concat(calls_data, ignore_index=True)

# Wandelt expiration in Ordinalzahl für Plot
df['expiration'] = pd.to_datetime(df['expiration'])
df['exp_num'] = df['expiration'].map(pd.Timestamp.toordinal)
df['impliedVolatility'] *= 100

# Grid erzeugen
pivot = df.pivot_table(index='strike', columns='exp_num', values='impliedVolatility')
pivot = pivot.interpolate(axis=0).interpolate(axis=1)  # Glätten von NaNs

X, Y = np.meshgrid(pivot.columns, pivot.index)
Z = pivot.values

# 3D Surface Plot
fig = go.Figure(data=[go.Surface(
    x=Y,  # Strike
    y=X,  # Verfall (ordinal)
    z=Z,  # Volatility
    colorscale='Greys',
    showscale=False,
    lighting=dict(ambient=0.8, diffuse=0.5, roughness=0.9),
    opacity=0.9
)])

# Clean Style
fig.update_layout(
    title='Implied Volatility Surface (MSFT Calls)',
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Expiration Date',
        zaxis_title='Implied Volatility (%)',
        xaxis=dict(showbackground=False, showgrid=False, zeroline=False),
        yaxis=dict(showbackground=False, showgrid=False, zeroline=False,
                   tickvals=[pd.to_datetime(d).toordinal() for d in df['expiration'].unique()],
                   ticktext=[d.strftime("%Y-%m-%d") for d in df['expiration'].unique()]),
        zaxis=dict(showbackground=False, showgrid=False, zeroline=False)
    ),
    template='plotly_white',
    height=800,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()

